In [1]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop, Adam
import os

Using TensorFlow backend.


In [2]:
batch_size = 128 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 10 # 訓練的 epochs 數量
BATCH_SIZE = [64,128,256]
EPOCHS = [5,10,50]

In [3]:
# 讀取資料並檢視
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print(f'x_train shape:{x_train.shape}')
print(f'{x_train.shape[0]} train samples')
print(f'{x_test.shape[0]} test samples')

# 對 label 進行 one-hot encoding (y_trian 原本是純數字)
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape:(50000, 32, 32, 3)
50000 train samples
10000 test samples


## 首先我們使用一般的 DNN (MLP) 來訓練
由於 DNN 只能輸入一維的資料，我們要先將影像進行攤平，若 (50000, 32, 32, 3) 的影像，攤平後會變成 (50000, 32*32*3) = (50000, 3072)

In [4]:
# 將資料攤平成一維資料
x_train = x_train.reshape(50000, 3072) 
x_test = x_test.reshape(10000, 3072)

# 將資料變為 float32 並標準化
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

50000 train samples
10000 test samples


In [5]:
dnn = Sequential()
dnn.add(Dense(512, activation='relu', input_shape=(3072,)))
dnn.add(Dropout(0.2))
dnn.add(Dense(512, activation='relu'))
dnn.add(Dropout(0.2))
dnn.add(Dense(num_classes, activation='softmax'))

dnn.summary()

dnn.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               1573376   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 1,841,162
Trainable params: 1,841,162
Non-trainable params: 0
_________________________________________________________________




In [6]:
DNN = {}

In [7]:
for batch_size in BATCH_SIZE:
    epochs = 10 # 訓練的 epochs 數量
    dnn.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
    DNN[f'b{str(batch_size)}'] = dnn.evaluate(x_test, y_test, verbose=0)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 50000 samples, validate on 10000 samples
Epoch 1/10





50000/50000 [==============================] - 11s 215us/step - loss: 2.1064 - acc: 0.2650 - val_loss: 1.8478 - val_acc: 0.3238
Epoch 2/10
50000/50000 [==============================] - 6s 110us/step - loss: 1.8314 - acc: 0.3377 - val_loss: 1.8778 - val_acc: 0.3085
Epoch 3/10
50000/50000 [==============================] - 6s 110us/step - loss: 1.7780 - acc: 0.3607 - val_loss: 1.7387 - val_acc: 0.3676
Epoch 4/10
50000/50000 [==============================] - 6s 111us/step - loss: 1.7378 - acc: 0.3757 - val_loss: 1.7214 - val_acc: 0.3943
Epoch 5/10
50000/50000 [==============================] - 6s 111us/step - loss: 1.7146 - acc: 0.3841 - val_loss: 1.6876 - val_acc: 0.3938
Epoch 6/10
50000/50000 [==============================] - 5s 109us/step - loss: 1.7011 - acc: 0.3907 - val_loss: 1.6982 - val_acc: 0.4032
Epoch 7/10
50000/50

In [8]:
for epochs in EPOCHS:
    batch_size = 128 # batch 的大小，如果出現 OOM error，請降低這個值
    dnn.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(x_test, y_test))
    DNN[f'e{str(epochs)}'] = dnn.evaluate(x_test, y_test, verbose=0)

Train on 50000 samples, validate on 10000 samples
Epoch 1/5
50000/50000 [==============================] - 3s 70us/step - loss: 1.5383 - acc: 0.4507 - val_loss: 1.5052 - val_acc: 0.4697
Epoch 2/5
50000/50000 [==============================] - 3s 70us/step - loss: 1.5288 - acc: 0.4539 - val_loss: 1.5017 - val_acc: 0.4807
Epoch 3/5
50000/50000 [==============================] - 4s 70us/step - loss: 1.5359 - acc: 0.4523 - val_loss: 1.5148 - val_acc: 0.4587
Epoch 4/5
50000/50000 [==============================] - 3s 70us/step - loss: 1.5274 - acc: 0.4550 - val_loss: 1.5149 - val_acc: 0.4629
Epoch 5/5
50000/50000 [==============================] - 3s 69us/step - loss: 1.5299 - acc: 0.4527 - val_loss: 1.5048 - val_acc: 0.4680
Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 3s 68us/step - loss: 1.5253 - acc: 0.4563 - val_loss: 1.5173 - val_acc: 0.4660
Epoch 2/10
50000/50000 [==============================] - 3s 68us/step - loss: 1.52

50000/50000 [==============================] - 4s 70us/step - loss: 1.4751 - acc: 0.4777 - val_loss: 1.5496 - val_acc: 0.4689
Epoch 45/50
50000/50000 [==============================] - 4s 70us/step - loss: 1.4796 - acc: 0.4762 - val_loss: 1.5206 - val_acc: 0.4652
Epoch 46/50
50000/50000 [==============================] - 4s 71us/step - loss: 1.4770 - acc: 0.4757 - val_loss: 1.5894 - val_acc: 0.4312
Epoch 47/50
50000/50000 [==============================] - 3s 68us/step - loss: 1.4790 - acc: 0.4766 - val_loss: 1.5255 - val_acc: 0.4579
Epoch 48/50
50000/50000 [==============================] - 3s 69us/step - loss: 1.4814 - acc: 0.4763 - val_loss: 1.5127 - val_acc: 0.4673
Epoch 49/50
50000/50000 [==============================] - 3s 70us/step - loss: 1.4744 - acc: 0.4760 - val_loss: 1.5251 - val_acc: 0.4734
Epoch 50/50
50000/50000 [==============================] - 4s 70us/step - loss: 1.4733 - acc: 0.4768 - val_loss: 1.4986 - val_acc: 0.4703


## 接下來我們使用 CNN 來訓練神經網路
CNN 的原理非常適合處理影像類的資料，就讓我們來看看，同樣的訓練條件，CNN 是否顯著優於 DNN 呢?

In [9]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [10]:
cnn = Sequential()
cnn.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
cnn.add(Activation('relu'))
cnn.add(Conv2D(32, (3, 3)))
cnn.add(Activation('relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2)))
cnn.add(Dropout(0.25))

cnn.add(Conv2D(64, (3, 3), padding='same'))
cnn.add(Activation('relu'))
cnn.add(Conv2D(64, (3, 3)))
cnn.add(Activation('relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2)))
cnn.add(Dropout(0.25))

cnn.add(Flatten())
cnn.add(Dense(512))
cnn.add(Activation('relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(num_classes))
cnn.add(Activation('softmax'))
cnn.summary()

cnn.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)        18496     
_________

In [11]:
CNN = {}

In [12]:
for batch_size in BATCH_SIZE:
    epochs = 10 # 訓練的 epochs 數量
    history = cnn.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
    CNN[f'b{str(batch_size)}'] = cnn.evaluate(x_test, y_test, verbose=0)

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 19s 373us/step - loss: 1.6197 - acc: 0.4119 - val_loss: 1.3490 - val_acc: 0.5251
Epoch 2/10
50000/50000 [==============================] - 16s 321us/step - loss: 1.1560 - acc: 0.5924 - val_loss: 1.0372 - val_acc: 0.6297
Epoch 3/10
50000/50000 [==============================] - 16s 318us/step - loss: 0.9846 - acc: 0.6571 - val_loss: 0.9103 - val_acc: 0.6804
Epoch 4/10
50000/50000 [==============================] - 16s 319us/step - loss: 0.8868 - acc: 0.6928 - val_loss: 1.2985 - val_acc: 0.5998
Epoch 5/10
50000/50000 [==============================] - 16s 315us/step - loss: 0.8334 - acc: 0.7114 - val_loss: 0.8425 - val_acc: 0.7173
Epoch 6/10
50000/50000 [==============================] - 16s 314us/step - loss: 0.7935 - acc: 0.7271 - val_loss: 1.0503 - val_acc: 0.6584
Epoch 7/10
50000/50000 [==============================] - 16s 314us/step - loss: 0.7741 - acc: 0.7343 - val_loss: 0.

In [13]:
for epochs in EPOCHS:
    batch_size = 128
    history = cnn.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
    CNN[f'e{str(epochs)}'] = cnn.evaluate(x_test, y_test, verbose=0)

Train on 50000 samples, validate on 10000 samples
Epoch 1/5
50000/50000 [==============================] - 13s 266us/step - loss: 0.5466 - acc: 0.8188 - val_loss: 0.6987 - val_acc: 0.7861
Epoch 2/5
50000/50000 [==============================] - 13s 266us/step - loss: 0.5613 - acc: 0.8152 - val_loss: 0.7138 - val_acc: 0.7654
Epoch 3/5
50000/50000 [==============================] - 13s 265us/step - loss: 0.5732 - acc: 0.8115 - val_loss: 0.6549 - val_acc: 0.7852
Epoch 4/5
50000/50000 [==============================] - 13s 263us/step - loss: 0.5705 - acc: 0.8148 - val_loss: 0.7039 - val_acc: 0.7777
Epoch 5/5
50000/50000 [==============================] - 13s 264us/step - loss: 0.5784 - acc: 0.8124 - val_loss: 0.7326 - val_acc: 0.7873
Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 13s 264us/step - loss: 0.5781 - acc: 0.8090 - val_loss: 0.6568 - val_acc: 0.7985
Epoch 2/10
50000/50000 [==============================] - 13s 263us/ste

50000/50000 [==============================] - 13s 261us/step - loss: 0.8793 - acc: 0.7250 - val_loss: 0.9156 - val_acc: 0.7177
Epoch 44/50
50000/50000 [==============================] - 13s 261us/step - loss: 0.8930 - acc: 0.7183 - val_loss: 0.8386 - val_acc: 0.7321
Epoch 45/50
50000/50000 [==============================] - 13s 260us/step - loss: 0.8975 - acc: 0.7203 - val_loss: 0.8529 - val_acc: 0.7289
Epoch 46/50
50000/50000 [==============================] - 13s 261us/step - loss: 0.9136 - acc: 0.7158 - val_loss: 0.7421 - val_acc: 0.7663
Epoch 47/50
50000/50000 [==============================] - 13s 259us/step - loss: 0.9191 - acc: 0.7141 - val_loss: 0.8035 - val_acc: 0.7359
Epoch 48/50
50000/50000 [==============================] - 13s 263us/step - loss: 0.9182 - acc: 0.7160 - val_loss: 0.7698 - val_acc: 0.7607
Epoch 49/50
50000/50000 [==============================] - 13s 261us/step - loss: 0.9362 - acc: 0.7107 - val_loss: 2.0077 - val_acc: 0.5728
Epoch 50/50
50000/50000 [=======

## DNN

In [14]:
DNN

{'b64': [0.8783127338409423, 0.7084],
 'b128': [0.6290167245864868, 0.7952],
 'b256': [0.6371337666034699, 0.8016],
 'e5': [0.7326333536624908, 0.7873],
 'e10': [0.6988567898750305, 0.782],
 'e50': [0.831518284702301, 0.7326]}

## CNN

In [15]:
CNN

{}

[loss, acc]

## 同樣運算 10 個 epochs，但 CNN 在 test data 的準確率顯著優於 DNN!

## 作業
1. 請試著調整各個超參數，並說明那些超參數對於結果有明顯的影響?

        epochs與batch_size皆與準確度呈正相關，
2. CNN 與 DNN 哪個模型的參數數量比較多? 造成參數的數量不同的原因在哪?

        DNN > CNN，主因是DNN為全連接，權重數量為(圖片尺寸*channels+1)*結點數；而CNN為(kernel_size*channels+1)*結點數。